In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("brabus61/joke-generator")
model = AutoModelForCausalLM.from_pretrained("brabus61/joke-generator", from_tf=True)

tokenizer_config.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/27.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/145k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/868 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/356M [00:00<?, ?B/s]

All TF 2.0 model weights were used when initializing GPT2LMHeadModel.

Some weights of GPT2LMHeadModel were not initialized from the TF 2.0 model and are newly initialized: ['lm_head.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
from datasets import load_dataset, get_dataset_split_names


dataset = load_dataset("json", data_files="/kaggle/input/anecdotsforproject/final_dataset.json", split="train")
dataset=dataset.train_test_split(test_size=0.2)
dataset

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'text'],
        num_rows: 4860
    })
    test: Dataset({
        features: ['id', 'text'],
        num_rows: 1215
    })
})

In [3]:
def tokenize(element):
    outputs = tokenizer(
        element["text"],
        truncation=True,
        max_length=128,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == 128:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}

In [4]:
tokenized_dataset = dataset.map(
    tokenize, batched=True, remove_columns=dataset["train"].column_names
)
tokenized_dataset

Map:   0%|          | 0/4860 [00:00<?, ? examples/s]

Map:   0%|          | 0/1215 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 2812
    })
    test: Dataset({
        features: ['input_ids'],
        num_rows: 661
    })
})

In [5]:
tokenized_dataset["train"][0]

{'input_ids': [33,
  1164,
  88,
  297,
  1553,
  262,
  453,
  12,
  394,
  258,
  1325,
  744,
  1214,
  12,
  293,
  789,
  1258,
  659,
  769,
  350,
  556,
  14,
  325,
  939,
  1948,
  282,
  326,
  265,
  3221,
  941,
  3975,
  769,
  350,
  556,
  296,
  265,
  1742,
  1022,
  1535,
  502,
  902,
  428,
  292,
  14,
  325,
  519,
  727,
  323,
  265,
  354,
  1123,
  389,
  73,
  350,
  556,
  12,
  748,
  1357,
  265,
  580,
  3184,
  3004,
  285,
  12,
  311,
  748,
  1357,
  487,
  457,
  12,
  1385,
  323,
  791,
  496,
  645,
  1202,
  3232,
  474,
  325,
  317,
  258,
  769,
  350,
  556,
  635,
  325,
  472,
  3106,
  89,
  287,
  386,
  635,
  325,
  55,
  861,
  12,
  1114,
  12,
  1505,
  296,
  841,
  323,
  470,
  270,
  1062,
  459,
  325,
  12,
  552,
  265,
  769,
  350,
  556,
  292,
  2008,
  12,
  325,
  53,
  78,
  2421,
  3915,
  12,
  367,
  487,
  3975,
  496,
  645,
  1202]}

In [6]:
from transformers import DataCollatorForLanguageModeling


data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [7]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
   output_dir="example_trainer",
    evaluation_strategy = "epoch", 
    learning_rate=2e-5,  
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=16, 
    num_train_epochs=3,
    weight_decay=0.01, 
    report_to = 'tensorboard',
    
)


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [8]:
trainer = Trainer(   
    model, 
    args,  
    train_dataset=tokenized_dataset["train"].select(range(1000)),  
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [9]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,nan
2,No log,nan
3,No log,nan


TrainOutput(global_step=189, training_loss=0.0, metrics={'train_runtime': 2701.7004, 'train_samples_per_second': 1.11, 'train_steps_per_second': 0.07, 'total_flos': 195969024000000.0, 'train_loss': 0.0, 'epoch': 3.0})

In [11]:
trainer.evaluate()

{'eval_loss': nan,
 'eval_runtime': 180.8763,
 'eval_samples_per_second': 3.654,
 'eval_steps_per_second': 0.232,
 'epoch': 3.0}

In [ ]:
import torch
from transformers import pipeline
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
pipe = pipeline(
    "text-generation", model="brabus61/joke-generator", device=device)
pipe("What kind of pig can you ignore at a party?", num_return_sequences=1[0]["generated_text"])